**<font size='5'>以下顺次点击下方每个单元格左侧的“运行”图标，不可跳过步骤</font>**
**</br>【重要】:** 务必在"修改"->"笔记本设置"->"硬件加速器"中选择GPU！否则处理速度会非常慢。
 **</br>【IMPORTANT】:** Make sure you select GPU as hardware accelerator in notebook settings, otherwise the processing speed will be very slow.

In [ ]:
#@title **通用参数/Required settings:**


# @markdown **【IMPORTANT】:**<font size="2">Select uploaded file type.
# @markdown **</br>【重要】:** 选择上传的文件类型(视频-video/音频-audio）</font>

# encoding:utf-8
file_type = "audio"  # @param ["audio","video"]

# @markdown #### **Youtube video or playlist**
yt_url = "https://www.youtube.com/watch?v=MVYrJJNdrEg"  # @param {type:"string"}

model_size = "large-v2"  # @param ["base","small","medium","medium.en", "large-v1","large-v2","large-v3"]
language = "en"  # @param {type:"string"}


In [ ]:
#@title **运行Whisper/Run Whisper**
#@markdown 完成后srt文件将自动下载到本地/srt file will be auto downloaded after finish.
! pip install faster-whisper
! pip install ffmpeg
! pip install yt_dlp
! pip install torch
! pip install pysubs2
from IPython.display import clear_output
clear_output()
print('语音识别库配置完毕，将开始转换')
import os
import subprocess
import yt_dlp
import torch
from google.colab import files
from faster_whisper import WhisperModel
from tqdm import tqdm
import time
import pandas as pd
import requests
from urllib.parse import quote_plus
from pathlib import Path
import sys
import pysubs2
import gc
import re

# assert file_name != ""
# assert language != ""
file_basenames = []

torch.cuda.empty_cache()
print('加载模型 Loading model...')
model = WhisperModel(model_size)

ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': '%(id)s.%(ext)s',
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
}


def sanitize_filename(title):
    # Remove invalid characters
    title = re.sub(r'[\\/*?:"<>|]', '', title)

    # Replace spaces with underscores
    title = title.replace(' ', '_')

    # Ensure the filename does not exceed the maximum length
    max_length = 255
    if len(title) > max_length:
        title = title[:max_length]

    return title

file_name = "audio.wav"
title = "no title"
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(yt_url)
    video_info = ydl.extract_info(yt_url, download=False)
    title = video_info['title']
    file_name = f"{video_info['id']}.wav"

file_basename = sanitize_filename(title)

# print(file_basename)
tic = time.time()
print('识别中 Transcribe in progress...')
segments, info = model.transcribe(audio = f'{file_name}',
                                    beam_size=5,
                                    language=language,
                                  initial_prompt="a sentence,")

# segments is a generator so the transcription only starts when you iterate over it
# to use pysubs2, the argument must be a segment list-of-dicts
total_duration = round(info.duration, 2)  # Same precision as the Whisper timestamps.
results= []
with tqdm(total=total_duration, unit=" seconds") as pbar:
    for s in segments:
        segment_dict = {'start':s.start,'end':s.end,'text':s.text}
        results.append(segment_dict)
        segment_duration = s.end - s.start
        pbar.update(segment_duration)


#Time comsumed
toc = time.time()
print('识别完毕 Done')
print(f'Time consumpution {toc-tic}s')

srt_file_path = file_basename+'.srt'
subs = pysubs2.load_from_whisper(results)
subs.save(srt_file_path)

files.download(srt_file_path)

torch.cuda.empty_cache()
print('字幕生成完毕 All done!')